In [13]:
import math
import random
import os

from collections import Counter

import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
from copy import copy
from scipy.optimize import minimize

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.metrics import confusion_matrix



In [14]:
# SPARSE REPRESENTATIONS - Q1

DATA_DIR = '../data/svm/data/'
POS_DATA_DIR = DATA_DIR + 'pos/'
NEG_DATA_DIR = DATA_DIR + 'neg/'
TRAIN_SPLIT_COUNT = 1500


In [15]:

# read review file into a list of strings
# each string = a token from the review
def read_file(filepath):
    f = open(filepath)
    lines = f.read().split(' ')
    symbols = '${}()[].,:;+-*/&|<>=~" '
    words = map(lambda Element: Element.translate(str.maketrans("", "", symbols)).strip(), lines)
    words = filter(None, words)
    return Counter(list(words))

# return list of file names in a directory
# skips directories
def list_dir_files(dirpath):
    file_names = []
    children = os.listdir(dirpath)

    for child in children:
        if os.path.isfile(os.path.join(dirpath, child)):
            file_names.append(child)
    
    return file_names

# read all files in the directory
def read_dir(dirpath):
    file_contents = []
    file_names = list_dir_files(dirpath)

    for file_name in file_names:
        filepath = os.path.join(dirpath, file_name)
        file_content = read_file(filepath)
        file_contents.append(file_content)

    return file_contents

In [16]:
pos_reviews = read_dir(POS_DATA_DIR)
neg_reviews = read_dir(NEG_DATA_DIR)

pos_review_points = list(zip(pos_reviews, [1] * len(pos_reviews)))
neg_review_points = list(zip(neg_reviews, [-1] * len(neg_reviews)))

all_points = pos_review_points.copy()
all_points.extend(neg_review_points)
random.shuffle(all_points)

train = all_points[:TRAIN_SPLIT_COUNT]
test = all_points[TRAIN_SPLIT_COUNT:]